# Clonamos el repositorio para obtener los dataSet

In [ ]:
!git clone https://github.com/joanby/tensorflow.git

# Damos acceso a nuestro Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Test it

In [ ]:
!ls '/content/drive/My Drive' 

# Google colab tools

In [ ]:
from google.colab import files # Para manejar los archivos y, por ejemplo, exportar a su navegador
import glob # Para manejar los archivos y, por ejemplo, exportar a su navegador
from google.colab import drive # Montar tu Google drive

##Especificando la versión de TensorFlow

Ejecutando "importar tensorflow" importará la versión por defecto (actualmente 2.x). Puedes usar la 1.x ejecutando una celda con la "versión mágica de tensorflow" **antes de ejecutar "importar tensorflow".

### Si no funciona hacer el pip install


In [ ]:
#!pip install tensorflow==1.14
%tensorflow_version 1.x

# Importar Tensorflow

In [ ]:
import tensorflow as tf
print(tf.__version__)
import matplotlib.pyplot as plt

In [ ]:
session = tf.Session()

# Long Short Term Memory

$$i_t = \sigma(B_ih_{t-1}+A_ix_t)$$

$$C_t = \tanh(B_ch_{t-1}+A_cx_t)$$

$$f_t = \sigma(B_fh_{t-1}+A_fx_t)$$

$$N_t = i_t\cdot C_t +f_t N_{t-1}$$

$$O_t = \sigma(B_oh_{t-1}+A_ox_t+D_oN_t)$$

$$h_t = O_t\cdot \tanh(N_t)$$

In [ ]:
import os
import re
import string
import requests
import numpy as np
import collections
import random
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
data_dir = "/content/tensorflow/datasets/shakespeare"
data_file = "shakespeare.txt"
model_path = "shakespeare_model"
full_model_dir = os.path.join(data_dir, model_path)

In [ ]:
string.punctuation

In [ ]:
punctuation = ''.join([x for x in string.punctuation if x not in ['-', "'"]])
punctuation

In [ ]:
if not os.path.exists(full_model_dir):
    os.makedirs(full_model_dir)

if not os.path.isfile(os.path.join(data_dir, data_file)):
    url = "http://www.gutenberg.org/cache/epub/100/pg100.txt"
    response = requests.get(url)
    s_text = response.content.decode('utf-8')
    s_text = s_text[7675:]
    s_text = s_text.replace("\r\n", '')
    s_text = s_text.replace("\n",'')
    
    with open(os.path.join(data_dir, data_file), "w") as file_out:
        file_out.write(s_text)
else:
    with open(os.path.join(data_dir, data_file), "r") as file_out:
        s_text = file_out.read().replace("\n",'')

In [ ]:
s_text = re.sub(r'[{}]'.format(punctuation), ' ', s_text)
s_text = re.sub('\s+', ' ', s_text).strip().lower()

In [ ]:
s_text

In [ ]:
session = tf.Session()

min_word_freq = 5
rnn_size = 128
epoch = 10
batch_size = 100
learning_rate = 0.001
training_seq_len = 50
embedding_size = rnn_size
save_every = 500
eval_every = 50

prime_texts = ['to be or not be', 'thou art more', 'wherefore art thou']

In [ ]:
def build_vocabulary(text, min_word_freq):
    word_count = collections.Counter(text.split(' '))
    word_count = {key:val for key, val in word_count.items() if val>min_word_freq}
    words = word_count.keys()
    word_to_vec = {key:(ix+1) for ix, key in enumerate(words)}
    word_to_vec['unknown'] = 0
    vec_to_word = {val:key for key, val in word_to_vec.items()}
    return vec_to_word, word_to_vec

In [ ]:
vec2word, word2vec = build_vocabulary(s_text, min_word_freq)

In [ ]:
vocab_size = len(vec2word)+1

In [ ]:
vocab_size

In [ ]:
s_text_words = s_text.split(' ')
s_text_vec = []
for ix, x in enumerate(s_text_words):
    try: 
        s_text_vec.append(word2vec[x])
    except:
        s_text_vec.append(0)
s_text_vec = np.array(s_text_vec)

In [ ]:
class LSTM_Model():
    def __init__(self, embedding_size, rnn_size, batch_size, learning_rate, ## embedding_size
                 training_seq_length, vocabulary_size, infer = False ):
        
        self.embedding_size = embedding_size ### guardar en self el objeto embedding_size
        
        self.rnn_size = rnn_size
        self.vocabulary_size = vocabulary_size
        self.infer = infer
        self.learning_rate = learning_rate
        
        if infer:
            self.batch_size = 1
            self.training_seq_len = 1
        else:
            self.batch_size = batch_size
            self.training_seq_len = training_seq_len
            
        self.lstm_cell = tf.contrib.rnn.BasicLSTMCell(self.rnn_size) ### cambiado por contrib!!!
        self.initial_state = self.lstm_cell.zero_state(self.batch_size, tf.float32)
        
        self.x_data = tf.placeholder(tf.int32, [self.batch_size, self.training_seq_len])
        self.y_output = tf.placeholder(tf.int32, [self.batch_size, self.training_seq_len])

        
        with tf.variable_scope('lstm_vars'):
            weigths = tf.get_variable('weigths', [self.rnn_size, self.vocabulary_size], tf.float32,
                                tf.random_normal_initializer())
            bias = tf.get_variable('bias', [self.vocabulary_size], tf.float32,
                                tf.constant_initializer(0.0))
            
            embedding_mat = tf.get_variable('embedding_mat', [self.vocabulary_size, self.embedding_size],
                                            tf.float32, tf.random_normal_initializer())
            
            embedding_output = tf.nn.embedding_lookup(embedding_mat, self.x_data)
            rnn_inputs = tf.split(axis=1, num_or_size_splits=self.training_seq_len, value=embedding_output)
            rnn_inputs_trimmed = [tf.squeeze(x, [1]) for x in rnn_inputs]
            
        def inferred_loop(prev, count):
            prev_trans = tf.add(tf.matmul(prev, weights), bias)
            prev_symbol = tf.stop_gradient(tf.argmax(prev_trans, 1))
            output = tf.nn.embedding_lookup(embedding_mat, prev_symbol)
            
        decoder = tf.contrib.legacy_seq2seq.rnn_decoder
        outputs, last_state = decoder(rnn_inputs_trimmed, ### FALTABA UNA S
                                      self.initial_state,
                                      self.lstm_cell,
                                      loop_function=inferred_loop if infer else None)
        
        output = tf.reshape(tf.concat(axis=1, values=outputs), [-1, self.rnn_size])
        
        self.logit_output = tf.add(tf.matmul(output, weigths),bias)
        self.model_output = tf.nn.softmax(self.logit_output)
        
        loss_func = tf.contrib.legacy_seq2seq.sequence_loss_by_example
        loss = loss_func([self.logit_output],[tf.reshape(self.y_output, [-1])],
                [tf.ones([self.batch_size * self.training_seq_len])],
                self.vocabulary_size)
        
        self.cost = tf.reduce_sum(loss) / (self.batch_size * self.training_seq_len)
        self.final_state = last_state

        gradients, _ = tf.clip_by_global_norm(tf.gradients(self.cost, tf.trainable_variables()), 4.5)
        
        optimizer = tf.train.AdamOptimizer(self.learning_rate)
        self.train_step = optimizer.apply_gradients(zip(gradients, tf.trainable_variables()))

        
        
    def sample(self, session, words = vec2word, vocab = word2vec, num = 10, prime_text = 'thou art'):
        state = session.run(self.lstm_cell.zero_state(1, tf.float32))
        word_list = prime_text.split()
        for word in word_list[:-1]:
            x = np.zeros((1,1))
            x[0,0] = vocab[word]
            feed_dict = {self.x_data: x, self.initial_state:state}
            [state] = session.run([self.final_state], feed_dict=feed_dict)
        
        out_sentence = prime_text
        word = word_list[-1]
        for n in range(num):
            x = np.zeros((1,1))
            x[0,0] = vocab[word]
            feed_dict = {self.x_data: x, self.initial_state: state}
            [model_output, state] = session.run([self.model_output, self.final_state], feed_dict=feed_dict)
            sample = np.argmax(model_output[0])
            if sample == 0:
                break
            word = words[sample]
            out_sentence = out_sentence + " " + word
        return out_sentence

In [ ]:
with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE) as scope:
    lstm_model = LSTM_Model(embedding_size, rnn_size, batch_size, learning_rate,
                           training_seq_len, vocab_size)
    scope.reuse_variables()
    test_lstm_model = LSTM_Model(embedding_size, rnn_size, batch_size, learning_rate, 
                                training_seq_len, vocab_size, infer=True)
    
    saver = tf.train.Saver()
    num_batches = int(len(s_text_vec)/(batch_size*training_seq_len))+1
    batches = np.array_split(s_text_vec, num_batches)
    batches = [np.resize(x, [batch_size, training_seq_len]) for x in batches]### había un for x in batch_size!!! 
    
    session.run(tf.global_variables_initializer())
    
    train_loss = []
    idx_count = 1
    
    for ep in range(epoch):
        random.shuffle(batches)
        targets = [np.roll(x, -1, axis=1) for x in batches]
        print("Epoch actual : "+str(ep+1)+ " / "+str(epoch))
        
        state = session.run(lstm_model.initial_state)
        for ix, batch in enumerate(batches):
            training_dict = {lstm_model.x_data: batch, lstm_model.y_output: targets[ix]}
            c, h = lstm_model.initial_state
            training_dict[c] = state.c
            training_dict[h] = state.h
            
            temp_loss, state, _ = session.run([lstm_model.cost, lstm_model.final_state, lstm_model.train_step],
                                             feed_dict = training_dict)
            train_loss.append(temp_loss)
            
            if idx_count % 10 == 0:
                summary = (idx_count, ep + 1, ix+1, num_batches+1, temp_loss)
                print("Iteración: {}, Epoch: {}, Batch: {} de {}, Pérdida: {:.2f}".format(*summary))
            if idx_count % save_every == 0:
                model_file_name = os.path.join(full_model_dir, "model")
                saver.save(session, model_file_name, global_step = idx_count)
                print("Modelo guardado correctamente en {}".format(model_file_name))
                
                dict_file = os.path.join(full_model_dir, 'vocab.pkl')
                with open(dict_file, 'wb') as dict_file_conn:
                    pickle.dump([word2vec, vec2word], dict_file_conn)
                    
            if idx_count % eval_every == 0:
                for sample in prime_texts:
                    print(test_lstm_model.sample(session, vec2word, word2vec, num = 10, prime_text = sample))
            
            idx_count += 1

In [ ]:
plt.plot(train_loss, 'k-')
plt.title("Pérdida secuencia a secuencia")
plt.xlabel("Iteración")
plt.ylabel("Pérdida")
plt.show()